# Using the data algebra for Statistics and Data Science

## Introduction

This is an intermediate level example of using the [data algebra](https://github.com/WinVector/data_algebra)
to translate a statistical method into an implementation that works both
with Pandas and in databases.

This example turns out to be fairly non-trivial, as it involves:

  * Aggregating data at two different granularities.
  * Combining the aggregations either through a "pass through an extend" or through "join results"
    strategy.
  * Re-shaping records to move from multi row records to single row records.

This can be daunting. The guiding principle is: decompose into sub-problems, solve those,
and then compose the pieces into a working solution. The data algebra supports this
strategy as it is optimized to build up data processing pipelines from smaller pieces
and is optimized for re-use and testing.

### The problem

We will demonstrate using the data algebra to solve a statistical problem: computing
a difference in means in terms of a pooled standard deviation. What makes it a challenge is: we
want to do this quickly per-group for possibly very many groups and re-scale by pooled standard
deviations to get a measure of effect sizes.

First we set up some example data in Python using Numpy and Pandas.

In [1]:
# import packages
import sqlite3
import numpy
import numpy.random
import pandas

from data_algebra.data_ops import *
from data_algebra.cdata import *
import data_algebra.SQLite

In [2]:
# build synthetic example data

# seed the pseudo-random generator for repeatability
numpy.random.seed(1999)

# choose our simulated number of observations
n_obs = 1000

d = pandas.DataFrame({
    'group': numpy.random.choice(['a', 'b', 'c'], size=n_obs, replace=True),
    'value': numpy.random.normal(0, 1, size=n_obs),
    'sensor': numpy.random.choice(['s1', 's2'], size=n_obs, replace=True),
})
# make the b group have an actual difference in means of s1 versus s2
group_b_sensor_s2_rows = (d['group'] == 'b') & (d['sensor'] == 's2')
d.loc[group_b_sensor_s2_rows, 'value'] = d.loc[group_b_sensor_s2_rows, 'value']  + 0.5

d.head()

,group,value,sensor
0,a,0.051306,s2
1,a,0.700005,s2
2,a,-1.022481,s2
3,b,1.862029,s1
4,c,-1.173817,s2


The data is synthetic. What is modeling is taking measurements from different groups using two different
sensors.


We want to see, for each group if the empirically observed
difference in means in the values recorded by sensor `s1` and sensor `s2` are different in an interesting way.
By our construction of the synthetic data there is a significant difference in group `b`, and not in any
other group.

This is essentially an [ANOVA](https://en.wikipedia.org/wiki/Analysis_of_variance)
and [T-test](https://en.wikipedia.org/wiki/Student%27s_t-test) type of question, where we define interesting as
the observed difference being rare under a null hypothesis such as the means and variance being shared
between `s1` and `s2` sensors per group.

Naturally there is statistical software that performs the above task well, but for our example let's pursue this
by hand. Our quantity of interest is going to be: for each group we want to
estimate `t = ((s1 estimate) - s2 estimate)) / (var(s1 estimate) + var(s2 estimate)).sqrt()`,
where `(si estimte) = mean(si)`. This estimate is using the fact that, for independent processes
variances are additive.

When `|t|` is large
(say 2 or 3), we consider the observed difference to be unlikely under the null hypothesis that the
true means or expected values of `s1` and `s2` sensor are identical per group.
The reasoning being under the null hypothesis
(and under fairly mild additional conditions and with enough data), `t` with be
nearly [Student-t distributed](https://en.wikipedia.org/wiki/Student%27s_t-distribution) where
absolute values as large as 2 or 3 being somewhat rare.

So let's estimate `t` using the data algebra.

## The Solution

The strategy is to break the calculation down into smaller solvable steps. The data algebra is essentially
a coding of [Codd's relational algebra](https://en.wikipedia.org/wiki/Relational_algebra) in Python. This is just
the thesis that if one learns a few primary data transforms, then many data processing tasks can
be effectively written in terms of these operations. The operations are typically:


   * Adding a column as function of other columns. That is for each row values are combined
      to create a new value. This often called an extension.
   * Computing an aggregation such as mean, max in one column controlled by a specification of
     which rows are to be grouped together. This is typically called projection if we want exactly
     one row result per group or a "window function" if we want one result row per input row.
   * Joining rows from two data frames that match on particular key columns. This is a powerful
     method of mapped lookup and cross-product formation.

In terms of these operators we want new columns such as `mean(s1 values)`, `mean(s2 values)`, and so on, to be calculated
per group. The organizing idea is:

> Imagine some columns such that if these columns were already in your data frame, then the calculation
> would be easy to finish. Then add these columns to your data frame.

Let's do that using the data algebra.

### Adding Some Columns

First we specify the operations we want to perform. The wish we are trying to satisfy is: "the calculation
would be much easier we already knew the pooled standard deviations and group sizes".
This is an extend operation partitioned by
our grouping column `group`.

Our definition of operators is as follows. We start with `describe_table()` which
build a description of the column structure of our data frame `d`.
We then call `.extend()` on this object to specify new columns (`group_sd` and `group_size`)
we want produced. The `partition_by` specifies which set of rows go into each calculation.
We also add more steps to combine these columns to get the per-group variances.
Notice the later steps don't use a partition to be specified, as we can
safely calculate per row. The rule is: each extend is separated to use only values that
are available before the step.

This is easiest just to see this in action.

In [3]:
# define our operators
ops = (
    describe_table(d, table_name='d')
        .extend(
            {
                'group_mean': 'value.mean()',
                'group_size': '(1).sum()'
            },
            partition_by=['group'])
        .extend({'sq_diff': '(value - group_mean)**2'})
        .extend(
            {'group_var': 'sq_diff.mean()'},
            partition_by=['group'])
        .extend({'group_var': 'group_var * group_size / (group_size - 1)'})  # adjust to sample variance
        .drop_columns(['sq_diff'])
    )

Each extend can only refer to values that already exist, this is to prevent
confusion as to which values are in which columns during calculation. The operations we used are:

  * `extend()`: add new columns to current rows. This can work either without a partition, where each calculation
     is performed among values in each row. Or this can work with a partition, where values are aggregated
     across groups of rows, but still written into the original rows. In an `extend()` the calculation
     is specified as a dictionary of new column values mapping to the quoted expressions to calculate the
     values. The expression grammar is similar to Python/Numpy/Pandas, with a good number of methods
     available.
  *  `drop_columns()`: deletes columns we no longer want.

The types of operators will be familiar to [dplyr]( https://CRAN.R-project.org/package=dplyr) users. However,
they originally come from Codd, and this style of emphasis on composition was
prototyped in [rqdatatable](https://CRAN.R-project.org/package=rqdatatable).

Our working values look like the following if we apply the operations we have up to now.

In [4]:
# apply our operators to our data frame d
ops.transform(d).head()

,group,value,sensor,group_mean,group_size,group_var
0,a,0.051306,s2,-0.032390,321,0.905921
1,a,0.700005,s2,-0.032390,321,0.905921
2,a,-1.022481,s2,-0.032390,321,0.905921
3,b,1.862029,s1,0.285289,324,0.973845
4,c,-1.173817,s2,0.041649,355,1.081075


Notice in the rows with `group == "a"` we see the same observed `group_var` (`0.905921`).

The `transform()` implementation is modular and is intended to eventually support other
realizations of Pandas style APIs.
Prospects include Dask, datatable, RAPIDS; but we have not started development on these adapters.
To support his prospect there is only one explicit reference to Pandas in the package, and
that reference can be overridden by the user.

### Adding Another Set of Columns

We also want the per `group` *and* `sensor` means to compare.
We can calculate this with a project, as we only want one output row for
each combination of `group` *and* `sensor`.

We continue our calculation as follows.

In [5]:
# define our operators
ops = (
    ops
        .project(
            {
                'group_sensor_mean': 'value.mean()',
                'group_sensor_size': '(1).sum()',
                'group_size': 'group_size.mean()',  # pseudo aggregation, value is constant in each group
                'group_var': 'group_var.mean()',    # pseudo aggregation, value is constant in each group
             },
            group_by=['group', 'sensor']
            )
        .extend({'group_sensor_est_var': 'group_var / group_sensor_size'})
        .drop_columns(['group_sensor_size', 'group_size', 'group_var'])
    )

The new operation we used is `project()`.

  * `project()` is a grouped operation where each group
     of rows is replaced by a single row. The grouping columns are copied into the new row, so
     they don't have to be specified. Any other columns must be created by calculations. The
     "pseudo aggregators" are how we copy-out columns we wish to keep around.


An alternative to
the `extend()` then `project()` solution we showed here would be to perform two projects and
then join the results back to together. We will illustrate this solution in an appendix.

Notice instead of applying new operations to our data frame, we instead append new
operations onto our existing operations pipeline `ops`.  This is the core of the data algebra:
operating on pipelines to produce larger re-usable pipelines.

We can examine our composite pipeline.

In [6]:
print(str(ops))

(
    TableDescription(table_name="d", column_names=["group", "value", "sensor"])
    .extend(
        {"group_mean": "value.mean()", "group_size": "(1).sum()"},
        partition_by=["group"],
    )
    .extend({"sq_diff": "(value - group_mean) ** 2"})
    .extend({"group_var": "sq_diff.mean()"}, partition_by=["group"])
    .extend({"group_var": "(group_var * group_size) / (group_size - 1)"})
    .drop_columns(["sq_diff"])
    .project(
        {
            "group_sensor_mean": "value.mean()",
            "group_sensor_size": "(1).sum()",
            "group_size": "group_size.mean()",
            "group_var": "group_var.mean()",
        },
        group_by=["group", "sensor"],
    )
    .extend({"group_sensor_est_var": "group_var / group_sensor_size"})
    .drop_columns(["group_sensor_size", "group_size", "group_var"])
)



And we can apply this pipeline to our data.


In [7]:
ops.transform(d)


,group,sensor,group_sensor_mean,group_sensor_est_var
0,a,s1,-0.103881,0.006761
1,a,s2,0.018839,0.004844
2,b,s1,0.097989,0.006049
3,b,s2,0.470291,0.005975
4,c,s1,0.069197,0.006512
5,c,s2,0.017453,0.005720


## Combining Rows to Get Results

We can now see the sensors seem to differ more in group `b` than in the other groups. Let's
finish the calculation and quantify this. We now have the issue of needing values from specific pairs of
rows. The simplest way to work around this is to get all the values we want into a single row and then work
forward.

What we mean is we wish to take a record that looks like the following.

In [8]:
a = pandas.DataFrame({
    'group': ['a', 'a'],
    'sensor': ['s1', 's2'],
    'group_sensor_mean': [-0.103881, 0.018839],
    'group_sensor_est_var': [0.006761, 0.004844],
})

a

,group,sensor,group_sensor_mean,group_sensor_est_var
0,a,s1,-0.103881,0.006761
1,a,s2,0.018839,0.004844


And transform it into a single row such as the following.

In [9]:
b = pandas.DataFrame({
    'group': ['a'],
    'group_sensor_mean_s1': [-0.103881],
    'group_sensor_mean_s2': [0.018839],
    'group_sensor_est_var_s1': [0.006761],
    'group_sensor_est_var_s2': [0.004844],
})

b

,group,group_sensor_mean_s1,group_sensor_mean_s2,group_sensor_est_var_s1,group_sensor_est_var_s2
0,a,-0.103881,0.018839,0.006761,0.004844


### Coordinatized Data

There are a number of ways to do this. Our preferred method is to use the [coordinatized data
methodology](https://github.com/WinVector/data_algebra/blob/main/Examples/cdata/cdata_general_example.ipynb).

In general the methodology works by specifying examples of the incoming and outgoing records, though
it does have convenience methods for common tasks such as melting, pivoting, and un-pivoting.

What we do is write down the incoming and outgoing record shapes.

In [10]:
record_in = pandas.DataFrame({
    'sensor': ['s1', 's2'],
    'group_sensor_mean': ['group_sensor_mean_s1', 'group_sensor_mean_s2'],
    'group_sensor_est_var': ['group_sensor_est_var_s1', 'group_sensor_est_var_s2'],
})

record_in


,sensor,group_sensor_mean,group_sensor_est_var
0,s1,group_sensor_mean_s1,group_sensor_est_var_s1
1,s2,group_sensor_mean_s2,group_sensor_est_var_s2


Notice these are just the example per-group records with specific values replaced by labels. These
examples then specify the transform. The convention is: single row records don't need to be specified, and
we draw out how multi row records work as follows.

In [11]:

record_map = RecordMap(
    blocks_in=data_algebra.cdata.RecordSpecification(
        control_table=record_in,
        record_keys=['group'],
    ),
)

And the transform essentially takes `a` to `b`.

In [12]:
record_map.transform(a)

,group,group_sensor_mean_s1,group_sensor_est_var_s1,group_sensor_mean_s2,group_sensor_est_var_s2
0,a,-0.103881,0.006761,0.018839,0.004844


We then add this step to our growing operator pipeline.

In [13]:
ops = (
    ops
        .convert_records(record_map)
)

ops.transform(d)

,group,group_sensor_mean_s1,group_sensor_est_var_s1,group_sensor_mean_s2,group_sensor_est_var_s2
0,a,-0.103881,0.006761,0.018839,0.004844
1,b,0.097989,0.006049,0.470291,0.005975
2,c,0.069197,0.006512,0.017453,0.005720


And we then finish our calculation using the available columns.

## Putting it all Together

In [14]:
ops = (
    ops
        .extend({'mean_diff': 'group_sensor_mean_s1 - group_sensor_mean_s2'})
        .extend({'t': 'mean_diff / (group_sensor_est_var_s1 + group_sensor_est_var_s2).sqrt()'})
        .drop_columns(['group_sensor_mean_s1', 'group_sensor_est_var_s1',
                       'group_sensor_mean_s2', 'group_sensor_est_var_s2'])
        .order_rows(['group'])
)

ops.transform(d)

,group,mean_diff,t
0,a,-0.122720,-1.139176
1,b,-0.372302,-3.395352
2,c,0.051744,0.467844


The new operations we showed here is 'order_rows()`, which re-orders rows. Because
data algebra is designed to work with SQL databases ordering (unless used to limit rows)
only is safe as a last step in a pipeline.

And we have our estimate: we reliably detect that the `b` is an unlikely measurement
if there were no per-sensor difference in means for this group. We can re-apply the `ops`
transform to any additional data frames that have the expected columns.

## Databases

An additional benefit of the data algebra is: the same operations can be applied to an arbitrary database
(currently Google BigQuery, PostgreSQL, MySQL, SQLite, and Spark).

To do this we build a model of the database connection.

In [15]:
sql_model = data_algebra.SQLite.SQLiteModel()
conn = sqlite3.connect(":memory:")
sql_model.prepare_connection(conn)
db_handle = data_algebra.db_model.DBHandle(db_model=sql_model, conn=conn)

We then, for purposes of illustration, insert our data into the database. In real applications
the data is usually already in the database.

In [16]:
db_handle.insert_table(d, table_name='d')

(TableDescription(table_name="d", column_names=["group", "value", "sensor"]))

And now we can build a table that contains the result:

In [17]:
db_handle.execute("CREATE TABLE res AS " + db_handle.to_sql(ops))

It is important to note, at this point all calculations are occurring in the database. No
data is round tripping between the database and Python. With the right database, this can achieve a performance and
scale that is beyond typical Python data frame tools and packages.

We can, of course, look at the result.

In [18]:
db_handle.read_query('SELECT * FROM res')

,group,mean_diff,t
0,a,-0.122720,-1.139176
1,b,-0.372302,-3.395352
2,c,0.051744,0.467844


Notice the results match the in-memory calculations

### SQL

The realized SQL can be quite long, but remember we were able to build up
our pipeline by composition. This allowed us to worry about each stage of operations
separately.

But, let's take a look at the produced SQL.

In [19]:
print(db_handle.to_sql(ops))

-- data_algebra SQL https://github.com/WinVector/data_algebra
--  dialect: SQLiteModel
--       string quote: '
--   identifier quote: "
WITH
 "convert_records_in_6" AS (
  SELECT  -- convert records
   "group_sensor_est_var_s2" ,
   "group_sensor_mean_s2" ,
   "group_sensor_est_var_s1" ,
   "group" ,
   "group_sensor_mean_s1"
  FROM
  (
  SELECT
 "group" AS "group",
 MAX(CASE WHEN  ( CAST("sensor" AS VARCHAR) = 's1' )  THEN "group_sensor_mean" ELSE NULL END) AS "group_sensor_mean_s1",
 MAX(CASE WHEN  ( CAST("sensor" AS VARCHAR) = 's1' )  THEN "group_sensor_est_var" ELSE NULL END) AS "group_sensor_est_var_s1",
 MAX(CASE WHEN  ( CAST("sensor" AS VARCHAR) = 's2' )  THEN "group_sensor_mean" ELSE NULL END) AS "group_sensor_mean_s2",
 MAX(CASE WHEN  ( CAST("sensor" AS VARCHAR) = 's2' )  THEN "group_sensor_est_var" ELSE NULL END) AS "group_sensor_est_var_s2"
FROM (
  SELECT  -- .extend({ 'group_sensor_est_var': 'group_var / group_sensor_size'})
 "group" ,
 "sensor" ,
 "group_sensor_mean" ,
 

The data algebra emits different SQL for different SQL dialects. Currently, we are actively developing
and testing on Google Big Query, PostgreSQL, MySQL, SQLite, and SparkSQL. However, adapting
to additional databases is a simple task.

## Conclusion

And that is how to translate a statistical calculation into a database using the data algebra.
All one needs to start is a list of the allowed operations and expressions, we have
links to such documentation [here](https://github.com/WinVector/data_algebra).

The data algebra is optimized to allow one to build up a data processing pipeline piece by piece.
This allows one to concentrate on solving sub-problems one at a time. The data algebra
emphasizes as operators acting on each other through composition, processing data is the
delayed end application.

The resulting data algebra transform pipeline can be
re-used on any number of data frames by the `.transform()` method and also used with different
databases using the `.to_sql()` method. Data algebra pipelines can be saved using standard Python pickling
procedures.


## Appendices

### The Entire Pipeline

In [20]:
print(ops)

(
    TableDescription(table_name="d", column_names=["group", "value", "sensor"])
    .extend(
        {"group_mean": "value.mean()", "group_size": "(1).sum()"},
        partition_by=["group"],
    )
    .extend({"sq_diff": "(value - group_mean) ** 2"})
    .extend({"group_var": "sq_diff.mean()"}, partition_by=["group"])
    .extend({"group_var": "(group_var * group_size) / (group_size - 1)"})
    .drop_columns(["sq_diff"])
    .project(
        {
            "group_sensor_mean": "value.mean()",
            "group_sensor_size": "(1).sum()",
            "group_size": "group_size.mean()",
            "group_var": "group_var.mean()",
        },
        group_by=["group", "sensor"],
    )
    .extend({"group_sensor_est_var": "group_var / group_sensor_size"})
    .drop_columns(["group_sensor_size", "group_size", "group_var"])
    .convert_records(
        data_algebra.cdata.RecordMap(
            blocks_in=data_algebra.cdata.RecordSpecification(
                record_keys=["group"],
      

### The Join Based Solution

Instead of chaining our partitioned `extend()`, and using pseudo-aggregators, we can
solve the problem with two `project()` steps pulled together with `natural_join()`.

In [21]:
ops_join = (
    describe_table(d, table_name='d')
        .extend({'group_mean': 'value.mean()'},
            partition_by=['group'])
        .extend({'sq_diff': '(value - group_mean)**2'})
        .project(
            {
                'group_var': 'sq_diff.mean()',
                'group_size': '(1).sum()',
            },
            group_by=['group'])
        .extend({'group_var': 'group_var * group_size / (group_size - 1)'})
        .natural_join(
            b=describe_table(d, table_name='d')
                .project(
                    {
                        'group_sensor_mean': 'value.mean()',
                        'group_sensor_size': '(1).sum()',
                    },
                    group_by=['group', 'sensor']
                    ),
            by=['group'],
            jointype='inner',
            )
        .extend({'group_sensor_est_var': 'group_var / group_sensor_size'})
        .convert_records(record_map)
        .extend({'mean_diff': 'group_sensor_mean_s1 - group_sensor_mean_s2'})
        .extend({'t': 'mean_diff / (group_sensor_est_var_s1 + group_sensor_est_var_s2).sqrt()'})
        .drop_columns(['group_sensor_mean_s1', 'group_sensor_est_var_s1',
                       'group_sensor_mean_s2', 'group_sensor_est_var_s2'])
        .order_rows(['group'])
    )

The join solution computes the required per-`['group']` summaries and per-`['group', 'sensor']` summaries
and then assembles them together. This replaces the "pass the first calculation through a partitioned
extend" strategy of the first solution. Notice our "pipeline" is actually a "DAG" (directed acyclic graph),
as we can use the same table as inputs in multiple places.

The new step here is the `natural_join()`.


  * `natural_join()` is a wrapper for Codd's fundamental relational operator:
    the join. A join takes two data frames and builds the cross product of every pair of rows (including
    some implicit special "missing" or "empty" rows) and retains the set of rows
    matching the join conditions. `natural_join()` adds some convenience, such as copying over
    join key columns and coalescing values for other column name collisions.


In our case the join matches all rows in our two projects, replicating rows from the result that is indexed only by
`['group']` so that every row indexed by `['group', 'sensor']` gets a match.
In our case the "`by`" and "`jointype`" arguments specify what set of rows to produce (or how to align results).

Notice we get the same results both in Pandas and from a database.



In [22]:
ops_join.transform(d)

,group,mean_diff,t
0,a,-0.122720,-1.139176
1,b,-0.372302,-3.395352
2,c,0.051744,0.467844


In [23]:
db_handle.read_query(ops_join)

,group,mean_diff,t
0,a,-0.122720,-1.139176
1,b,-0.372302,-3.395352
2,c,0.051744,0.467844


## Clean Up

In [24]:
db_handle.close()  # clean up